# Librerias a importar

In [2]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import sklearn
import os
import time

2025-09-26 02:25:28.753476: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-26 02:25:28.842709: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-26 02:25:30.808604: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras import layers
from keras import initializers
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

## Se instalan las dependecias si no las tenemos

In [5]:
# !pip3 install --upgrade wandb
#!pip install scikit-learn

## Nos registramos en WANDB para guardar los experimentos

In [289]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: luisedu04 (luisedu04-buap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Se definen los híper.parámetros a usar

In [290]:
lr = 0.005 #Learning rate
epochs = 50
bs = 120 #batch size
loss = "mse"
optimizer = "ADAM"
regularizer = "L2"

In [291]:
run = wandb.init(
    settings=wandb.Settings(x_disable_stats=False, x_stats_sampling_interval = 1.0),
    # set the wandb project where this run will be logged
    name = "Intento_intento_con_DO_L1L2_adam_more_data",
    project = "Creacion_de_regresion_lineal",
    group = "Pol",

    # track hyperparameters and run metadata with wandb.config
    config = {
        "layer_1": 512,
        "activation": "relu",
        "w_initializer": "He Initialization",
        "optimizer": optimizer,
        "loss": loss,
        "metric": "mse",
        "epoch": epochs,
        "batch_size": bs,
        "layer_quantity": 1,
        "regularizer": regularizer,
        "Modelo": "Pol"
    }
)
time.sleep(3.0)

# Preparando la red neuronal (Recolección de datos y preprocesamiento)

## Definimos las funciones que queremos usar

In [7]:
domain = np.linspace(-1,1,20000)

def function_1(domain):
    return 3*np.sin(np.pi * domain)

def function_2(domain):
    return 1 + 2*domain + 4*(domain**3)

codomain_sin = function_1(domain)
codomain_pol = function_2(domain)

#Transformamos los arreglos de numpy en vectores para que keras los reconozca
domain = domain.reshape(-1,1)
codomain_sin = codomain_sin.reshape(-1,1)
codomain_pol = codomain_pol.reshape(-1,1)

array([[-1.        ],
       [-0.99989999],
       [-0.99979999],
       ...,
       [ 0.99979999],
       [ 0.99989999],
       [ 1.        ]], shape=(20000, 1))

## Creamos los datos que va a recibir la red neuronal

In [293]:
#Se dividen los datos en train y test de tipo 80:20 (train:test) y se deja el random state fijo para replicabilidad

X_train_sin, X_test_sin, y_train_sin, y_test_sin = train_test_split(domain, codomain_sin, test_size=0.2, random_state=42)

X_train_pol, X_test_pol, y_train_pol, y_test_pol = train_test_split(domain, codomain_pol, test_size=0.2, random_state=57) 

## Escalamos los datos (estandarización)

In [294]:
scale_x_sin = StandardScaler()
scale_y_sin = StandardScaler()
X_train_sin, y_train_sin = scale_x_sin.fit_transform(X_train_sin), scale_y_sin.fit_transform(y_train_sin)
X_test_sin, y_test_sin = scale_x_sin.transform(X_test_sin), scale_y_sin.transform(y_test_sin)

scale_x_pol = StandardScaler()
scale_y_pol = StandardScaler()
X_train_pol, y_train_pol = scale_x_pol.fit_transform(X_train_pol), scale_y_pol.fit_transform(y_train_pol)
X_train_pol, y_train_pol = scale_x_pol.transform(X_train_pol), scale_y_pol.transform(y_train_pol)

# Creación de la red neuronal

In [295]:
entrada = 1
stddev1 = np.sqrt(2/entrada)
stddev2 = np.sqrt(2/256)
stddev3 = np.sqrt(2/128)
stddev4 = np.sqrt(2/64)

model_pol_data = keras.Sequential(
    [
        layers.Dense(256, activation = "relu", name = "layer1", 
                     kernel_initializer=initializers.RandomNormal(stddev = stddev1),
                     kernel_regularizer=regularizers.L2(l2=1e-5),
                    ),
        Dropout(0.2),        
        layers.Dense(128, activation = "relu", name = "layer2", 
                     kernel_initializer=initializers.RandomNormal(stddev = stddev2),
                     kernel_regularizer=regularizers.L2(l2=1e-5),
                    ),
        Dropout(0.2),
        layers.Dense(64, activation = "relu", name = "layer3", 
                     kernel_initializer=initializers.RandomNormal(stddev = stddev3),
                     kernel_regularizer=regularizers.L2(l2=1e-5),
                    ),
        Dropout(0.2),
        layers.Dense(32, activation = "relu", name = "layer4", 
                     kernel_initializer=initializers.RandomNormal(stddev = stddev3),
                     kernel_regularizer=regularizers.L2(l2=1e-5),
                    ),
        Dropout(0.2),
        #layers.Dense(16, activation="tanh", name="layer5"),
        layers.Dense(1) #Garantizamos una F(x) = y ya que solo queremos un valor posible para y
    ]
) #Arquitectura de la red neuronal

model_pol_data.build(input_shape=(None, entrada))
model_pol_data.compile(loss = loss, optimizer = Adam(learning_rate = lr),metrics = ["mse"]) #Función de costo
model_pol_data.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 256)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer3 (Dense)                  │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer4 (Dense)                  │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,777 (171.00 KB)

 Trainable params: 43,777 (171.00 KB)

 Non-trainable params: 0 (0.00 B)

## Reset ANN if needed

In [296]:
# for layer in model_pol_data.layers:
#    if hasattr(layer, 'kernel_initializer'):
#        new_weights = layer.kernel_initializer(layer.kernel.shape)
#        layer.kernel.assign(new_weights)
#    if hasattr(layer, 'bias_initializer') and layer.bias is not None:
#        new_bias = layer.bias_initializer(layer.bias.shape)
#        layer.bias.assign(new_bias)

## Model History

In [297]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model_pol_data.fit(X_train_pol, y_train_pol,
                    batch_size = bs,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (X_test_pol, y_test_pol),
                    callbacks=[
                        WandbMetricsLogger(log_freq=1),
                        WandbModelCheckpoint("models/model_pol_data.keras")
                    ]
                            )
run.finish()

Epoch 1/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.3408 - mse: 0.3314 - val_loss: 7.7838 - val_mse: 7.7745
Epoch 2/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0445 - mse: 0.0354 - val_loss: 8.0616 - val_mse: 8.0525
Epoch 3/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0341 - mse: 0.0251 - val_loss: 8.1890 - val_mse: 8.1800
Epoch 4/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0312 - mse: 0.0222 - val_loss: 8.3504 - val_mse: 8.3415
Epoch 5/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0290 - mse: 0.0201 - val_loss: 8.3995 - val_mse: 8.3906
Epoch 6/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0274 - mse: 0.0186 - val_loss: 8.4647 - val_mse: 8.4559
Epoch 7/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0263 - mse: 0.0175 - val_loss: 8.4699 - val_mse: 8.4612
Epoch 8/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0246 - mse: 0.0159 - val_loss: 8.5616 - val_mse: 8.5530
Epoch 9/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - lo

batch/batch_step,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mse,█▇▇▆▆▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/mse,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,▁▃▄▆▆▇▇▇▇▇███▇██████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
epoch/val_mse,▁▃▄▆▆▇▇▇▇▇███▇█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
batch/batch_step,6699


# Graficamos las funciones

In [298]:
# Value = np.linspace(-1,1, 2000).reshape(-1,1)
# a = model_sin.predict(Value)
# a = scale_y_sin.inverse_transform(a)

# plt.plot(Value, a, label = "Prediccón")
# plt.plot(Value, function_1(Value), label = "3sin(pi * x)")
# plt.legend()
# plt.show

In [8]:
Value = np.linspace(-1,1, 1000).reshape(-1,1)
b = model_pol_data.predict(Value)
b = scale_y_pol.inverse_transform(b)

plt.plot(Value, b, label = "Prediccón")
plt.plot(domain, codomain_pol, label = "1 + 2x + 4x^2")
plt.legend()
plt.show

NameError: name 'model' is not defined